# Library

In [ ]:
# !pip install arch
# !pip install tqdm

In [3]:
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
import itertools
from tqdm import tqdm

from scipy import stats
import statsmodels.api as sm
import arch

# Logic Generator for Extracting Data
**[스키마: 구단위 레벨로 지수 존재]**  
전국 수도권(서울/경기/인천) 6대광역시(부산/대구/광주/대전/울산/인천) 5대광역시(부산/대구/광주/대전/울산) 기타지방(나머지)  
서울 강남 강북 경기 인천 부산 대구 광주 대전 울산  
강원 충북 충남 전북 전남 경북 경남 제주  

**[선별: 구단위 제외한 지수 중 전국지역]**  
서울 강북 강남 6대광역시 경기 강원 충북 충남 전북 전남 경북 경남 (제주) 기타지방(수도권광역시제외지역)  

**[선별: 구단위 제외한 값으로 아파트/단독/연립 모두 존재 지역]**  
서울 강북 강남 6개광역시 경기 기타지방(수도권광역시제외지역)  
(강원 충북 충남 전북 전남 경북 경남은 아파트만 값 존재)  

In [4]:
# Calculate Logic of Data Extraction from KB Housing File
def Extracting_IndexValue(type_names, region_names, sheet_names, selected_names, sheet_val, col_val):
    col_names = ['Time']
    for combination in itertools.product(region_names, type_names):
        col_names.append(combination[0]+':'+combination[1])

    col_logic = ['='+sheet_names[0]+'!A5']
    for combination in itertools.product(selected_names, sheet_names):
        col_logic.append('='+combination[1]+'!'+combination[0])
    index_val = pd.DataFrame(col_logic, index=col_names).T

    val_index = []
    for combination in itertools.product(sheet_val, col_val):
        val_index.append('='+combination[0]+'!'+combination[1])
    value = pd.DataFrame(val_index, index=col_names[1:]).T
    return index_val, value

type_names = ['Apartments(HPI)', 'Detached_Houses(HPI)', 'Row_Houses(HPI)']
region_names = ['Seoul', 'Northern Seoul', 'Southern Seoul', '6 Large Cities', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do', 
                'Jeollabuk-do', 'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Non-Metropolitan Area']
sheet_names = ['매매APT', '매매단독', '매매연립']
selected_names = ['C5', 'D5', 'S5', 'AE5', 'CJ5', 'EE5', 'EI5', 'EQ5', 'FA5', 'FG5', 'FL5', 'FS5', 'GG5']
sheet_val = ['평균매매']
col_val = ['I125', 'J125', 'K125', 'M125', 'N125', 'O125', 'Q125', 'R125', 'S125', 'U125', 'V125', 'W125', 'AU125', 'AV125', 'AW125', 
               'AY125', '_125', '_125', 'BA125', '_125', '_125', 'BC125', '_125', '_125', 'BE125', '_125', '_125', 'BG125', '_125', '_125', 
               'BI125', '_125', '_125', 'BK125', '_125', '_125', 'BM125', 'BN125', 'BO125']
HPI_index, HPI_value = Extracting_IndexValue(type_names, region_names, sheet_names, selected_names, sheet_val, col_val)

type_names = ['Apartments(JDI)', 'Detached_Houses(JDI)', 'Row_Houses(JDI)']
region_names = ['Seoul', 'Northern Seoul', 'Southern Seoul', '6 Large Cities', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do', 
                'Jeollabuk-do', 'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Non-Metropolitan Area']
sheet_names = ['전세APT', '전세단독', '전세연립']
selected_names = ['C5', 'D5', 'S5', 'AE5', 'CJ5', 'EE5', 'EI5', 'EQ5', 'FA5', 'FG5', 'FL5', 'FS5', 'GG5']
sheet_val = ['평균전세']
col_val = ['I95', 'J95', 'K95', 'M95', 'N95', 'O95', 'Q95', 'R95', 'S95', 'U95', 'V95', 'W95', 'AU95', 'AV95', 'AW95', 
               'AY95', '_95', '_95', 'BA95', '_95', '_95', 'BC95', '_95', '_95', 'BE95', '_95', '_95', 'BG95', '_95', '_95', 
               'BI95', '_95', '_95', 'BK95', '_95', '_95', 'BM95', 'BN95', 'BO95']
JDI_index, JDI_value = Extracting_IndexValue(type_names, region_names, sheet_names, selected_names, sheet_val, col_val)
display(HPI_index)
display(HPI_value)
display(JDI_index)
display(JDI_value)

,Time,Seoul:Apartments(HPI),Seoul:Detached_Houses(HPI),Seoul:Row_Houses(HPI),Northern Seoul:Apartments(HPI),Northern Seoul:Detached_Houses(HPI),Northern Seoul:Row_Houses(HPI),Southern Seoul:Apartments(HPI),Southern Seoul:Detached_Houses(HPI),Southern Seoul:Row_Houses(HPI),6 Large Cities:Apartments(HPI),6 Large Cities:Detached_Houses(HPI),6 Large Cities:Row_Houses(HPI),Gyeonggi-do:Apartments(HPI),Gyeonggi-do:Detached_Houses(HPI),Gyeonggi-do:Row_Houses(HPI),Gangwon-do:Apartments(HPI),Gangwon-do:Detached_Houses(HPI),Gangwon-do:Row_Houses(HPI),Chungcheongbuk-do:Apartments(HPI),Chungcheongbuk-do:Detached_Houses(HPI),Chungcheongbuk-do:Row_Houses(HPI),Chungcheongnam-do:Apartments(HPI),Chungcheongnam-do:Detached_Houses(HPI),Chungcheongnam-do:Row_Houses(HPI),Jeollabuk-do:Apartments(HPI),Jeollabuk-do:Detached_Houses(HPI),Jeollabuk-do:Row_Houses(HPI),Jeollanam-do:Apartments(HPI),Jeollanam-do:Detached_Houses(HPI),Jeollanam-do:Row_Houses(HPI),Gyeongsangbuk-do:Apartments(HPI),Gyeongsangbuk-do:Detached_Houses(HPI),Gyeongsangbuk-do:Row_Houses(HPI),Gyeongsangnam-do:Apartments(HPI),Gyeongsangnam-do:Detached_Houses(HPI),Gyeongsangnam-do:Row_Houses(HPI),Non-Metropolitan Area:Apartments(HPI),Non-Metropolitan Area:Detached_Houses(HPI),Non-Metropolitan Area:Row_Houses(HPI)
0,=매매APT!A5,=매매APT!C5,=매매단독!C5,=매매연립!C5,=매매APT!D5,=매매단독!D5,=매매연립!D5,=매매APT!S5,=매매단독!S5,=매매연립!S5,=매매APT!AE5,=매매단독!AE5,=매매연립!AE5,=매매APT!CJ5,=매매단독!CJ5,=매매연립!CJ5,=매매APT!EE5,=매매단독!EE5,=매매연립!EE5,=매매APT!EI5,=매매단독!EI5,=매매연립!EI5,=매매APT!EQ5,=매매단독!EQ5,=매매연립!EQ5,=매매APT!FA5,=매매단독!FA5,=매매연립!FA5,=매매APT!FG5,=매매단독!FG5,=매매연립!FG5,=매매APT!FL5,=매매단독!FL5,=매매연립!FL5,=매매APT!FS5,=매매단독!FS5,=매매연립!FS5,=매매APT!GG5,=매매단독!GG5,=매매연립!GG5


,Seoul:Apartments(HPI),Seoul:Detached_Houses(HPI),Seoul:Row_Houses(HPI),Northern Seoul:Apartments(HPI),Northern Seoul:Detached_Houses(HPI),Northern Seoul:Row_Houses(HPI),Southern Seoul:Apartments(HPI),Southern Seoul:Detached_Houses(HPI),Southern Seoul:Row_Houses(HPI),6 Large Cities:Apartments(HPI),6 Large Cities:Detached_Houses(HPI),6 Large Cities:Row_Houses(HPI),Gyeonggi-do:Apartments(HPI),Gyeonggi-do:Detached_Houses(HPI),Gyeonggi-do:Row_Houses(HPI),Gangwon-do:Apartments(HPI),Gangwon-do:Detached_Houses(HPI),Gangwon-do:Row_Houses(HPI),Chungcheongbuk-do:Apartments(HPI),Chungcheongbuk-do:Detached_Houses(HPI),Chungcheongbuk-do:Row_Houses(HPI),Chungcheongnam-do:Apartments(HPI),Chungcheongnam-do:Detached_Houses(HPI),Chungcheongnam-do:Row_Houses(HPI),Jeollabuk-do:Apartments(HPI),Jeollabuk-do:Detached_Houses(HPI),Jeollabuk-do:Row_Houses(HPI),Jeollanam-do:Apartments(HPI),Jeollanam-do:Detached_Houses(HPI),Jeollanam-do:Row_Houses(HPI),Gyeongsangbuk-do:Apartments(HPI),Gyeongsangbuk-do:Detached_Houses(HPI),Gyeongsangbuk-do:Row_Houses(HPI),Gyeongsangnam-do:Apartments(HPI),Gyeongsangnam-do:Detached_Houses(HPI),Gyeongsangnam-do:Row_Houses(HPI),Non-Metropolitan Area:Apartments(HPI),Non-Metropolitan Area:Detached_Houses(HPI),Non-Metropolitan Area:Row_Houses(HPI)
0,=평균매매!I125,=평균매매!J125,=평균매매!K125,=평균매매!M125,=평균매매!N125,=평균매매!O125,=평균매매!Q125,=평균매매!R125,=평균매매!S125,=평균매매!U125,=평균매매!V125,=평균매매!W125,=평균매매!AU125,=평균매매!AV125,=평균매매!AW125,=평균매매!AY125,=평균매매!_125,=평균매매!_125,=평균매매!BA125,=평균매매!_125,=평균매매!_125,=평균매매!BC125,=평균매매!_125,=평균매매!_125,=평균매매!BE125,=평균매매!_125,=평균매매!_125,=평균매매!BG125,=평균매매!_125,=평균매매!_125,=평균매매!BI125,=평균매매!_125,=평균매매!_125,=평균매매!BK125,=평균매매!_125,=평균매매!_125,=평균매매!BM125,=평균매매!BN125,=평균매매!BO125


,Time,Seoul:Apartments(JDI),Seoul:Detached_Houses(JDI),Seoul:Row_Houses(JDI),Northern Seoul:Apartments(JDI),Northern Seoul:Detached_Houses(JDI),Northern Seoul:Row_Houses(JDI),Southern Seoul:Apartments(JDI),Southern Seoul:Detached_Houses(JDI),Southern Seoul:Row_Houses(JDI),6 Large Cities:Apartments(JDI),6 Large Cities:Detached_Houses(JDI),6 Large Cities:Row_Houses(JDI),Gyeonggi-do:Apartments(JDI),Gyeonggi-do:Detached_Houses(JDI),Gyeonggi-do:Row_Houses(JDI),Gangwon-do:Apartments(JDI),Gangwon-do:Detached_Houses(JDI),Gangwon-do:Row_Houses(JDI),Chungcheongbuk-do:Apartments(JDI),Chungcheongbuk-do:Detached_Houses(JDI),Chungcheongbuk-do:Row_Houses(JDI),Chungcheongnam-do:Apartments(JDI),Chungcheongnam-do:Detached_Houses(JDI),Chungcheongnam-do:Row_Houses(JDI),Jeollabuk-do:Apartments(JDI),Jeollabuk-do:Detached_Houses(JDI),Jeollabuk-do:Row_Houses(JDI),Jeollanam-do:Apartments(JDI),Jeollanam-do:Detached_Houses(JDI),Jeollanam-do:Row_Houses(JDI),Gyeongsangbuk-do:Apartments(JDI),Gyeongsangbuk-do:Detached_Houses(JDI),Gyeongsangbuk-do:Row_Houses(JDI),Gyeongsangnam-do:Apartments(JDI),Gyeongsangnam-do:Detached_Houses(JDI),Gyeongsangnam-do:Row_Houses(JDI),Non-Metropolitan Area:Apartments(JDI),Non-Metropolitan Area:Detached_Houses(JDI),Non-Metropolitan Area:Row_Houses(JDI)
0,=전세APT!A5,=전세APT!C5,=전세단독!C5,=전세연립!C5,=전세APT!D5,=전세단독!D5,=전세연립!D5,=전세APT!S5,=전세단독!S5,=전세연립!S5,=전세APT!AE5,=전세단독!AE5,=전세연립!AE5,=전세APT!CJ5,=전세단독!CJ5,=전세연립!CJ5,=전세APT!EE5,=전세단독!EE5,=전세연립!EE5,=전세APT!EI5,=전세단독!EI5,=전세연립!EI5,=전세APT!EQ5,=전세단독!EQ5,=전세연립!EQ5,=전세APT!FA5,=전세단독!FA5,=전세연립!FA5,=전세APT!FG5,=전세단독!FG5,=전세연립!FG5,=전세APT!FL5,=전세단독!FL5,=전세연립!FL5,=전세APT!FS5,=전세단독!FS5,=전세연립!FS5,=전세APT!GG5,=전세단독!GG5,=전세연립!GG5


,Seoul:Apartments(JDI),Seoul:Detached_Houses(JDI),Seoul:Row_Houses(JDI),Northern Seoul:Apartments(JDI),Northern Seoul:Detached_Houses(JDI),Northern Seoul:Row_Houses(JDI),Southern Seoul:Apartments(JDI),Southern Seoul:Detached_Houses(JDI),Southern Seoul:Row_Houses(JDI),6 Large Cities:Apartments(JDI),6 Large Cities:Detached_Houses(JDI),6 Large Cities:Row_Houses(JDI),Gyeonggi-do:Apartments(JDI),Gyeonggi-do:Detached_Houses(JDI),Gyeonggi-do:Row_Houses(JDI),Gangwon-do:Apartments(JDI),Gangwon-do:Detached_Houses(JDI),Gangwon-do:Row_Houses(JDI),Chungcheongbuk-do:Apartments(JDI),Chungcheongbuk-do:Detached_Houses(JDI),Chungcheongbuk-do:Row_Houses(JDI),Chungcheongnam-do:Apartments(JDI),Chungcheongnam-do:Detached_Houses(JDI),Chungcheongnam-do:Row_Houses(JDI),Jeollabuk-do:Apartments(JDI),Jeollabuk-do:Detached_Houses(JDI),Jeollabuk-do:Row_Houses(JDI),Jeollanam-do:Apartments(JDI),Jeollanam-do:Detached_Houses(JDI),Jeollanam-do:Row_Houses(JDI),Gyeongsangbuk-do:Apartments(JDI),Gyeongsangbuk-do:Detached_Houses(JDI),Gyeongsangbuk-do:Row_Houses(JDI),Gyeongsangnam-do:Apartments(JDI),Gyeongsangnam-do:Detached_Houses(JDI),Gyeongsangnam-do:Row_Houses(JDI),Non-Metropolitan Area:Apartments(JDI),Non-Metropolitan Area:Detached_Houses(JDI),Non-Metropolitan Area:Row_Houses(JDI)
0,=평균전세!I95,=평균전세!J95,=평균전세!K95,=평균전세!M95,=평균전세!N95,=평균전세!O95,=평균전세!Q95,=평균전세!R95,=평균전세!S95,=평균전세!U95,=평균전세!V95,=평균전세!W95,=평균전세!AU95,=평균전세!AV95,=평균전세!AW95,=평균전세!AY95,=평균전세!_95,=평균전세!_95,=평균전세!BA95,=평균전세!_95,=평균전세!_95,=평균전세!BC95,=평균전세!_95,=평균전세!_95,=평균전세!BE95,=평균전세!_95,=평균전세!_95,=평균전세!BG95,=평균전세!_95,=평균전세!_95,=평균전세!BI95,=평균전세!_95,=평균전세!_95,=평균전세!BK95,=평균전세!_95,=평균전세!_95,=평균전세!BM95,=평균전세!BN95,=평균전세!BO95


# Summary of Function

In [5]:
# Summary Statistics
def summary_statistics(Target, lag_candi1 = 10, lag_candi2 = 20):
    ks_stat_all, ks_pvalue_all, jb_stat_all, jb_pvalue_all = [], [], [], []
    arch_stat_all_lag1, arch_pvalue_all_lag1, arch_stat_all_lag2, arch_pvalue_all_lag2 = [], [], [], []
    lb_stat_all_lag1, lb_pvalue_all_lag1, lb_stat_all_lag2, lb_pvalue_all_lag2 = [], [], [], []
    lb_stat_all_lag12, lb_pvalue_all_lag12, lb_stat_all_lag22, lb_pvalue_all_lag22 = [], [], [], []
    adf_stat_all, adf_pvalue_all = [], []
    for col in Target.columns:
        # Kolmogorov–Smirnov Test
        ks_stat, ks_pvalue = stats.kstest(Target[col], cdf='norm', alternative='two-sided', mode='asymp')
        ks_stat_all.append(ks_stat)
        ks_pvalue_all.append(ks_pvalue)
        # Jarque–Bera Test
        jb_stat, jb_pvalue = stats.jarque_bera(Target[col])
        jb_stat_all.append(jb_stat)
        jb_pvalue_all.append(jb_pvalue)
        # ARCH Test
        arch_stat, arch_pvalue, arch_fstat, arch_fpvalue = sm.stats.diagnostic.het_arch(Target[col], maxlag=lag_candi1)
        arch_stat_all_lag1.append(arch_stat)
        arch_pvalue_all_lag1.append(arch_pvalue)
        arch_stat, arch_pvalue, arch_fstat, arch_fpvalue = sm.stats.diagnostic.het_arch(Target[col], maxlag=lag_candi2)
        arch_stat_all_lag2.append(arch_stat)
        arch_pvalue_all_lag2.append(arch_pvalue)
        # Ljung-Box Test
        lb_stat, lb_pvalue = sm.stats.diagnostic.acorr_ljungbox(Target[col], lags=lag_candi1)
        lb_stat, lb_pvalue = lb_stat[lag_candi1-1], lb_pvalue[lag_candi1-1]
        lb_stat_all_lag1.append(lb_stat)
        lb_pvalue_all_lag1.append(lb_pvalue)
        lb_stat, lb_pvalue = sm.stats.diagnostic.acorr_ljungbox(Target[col], lags=lag_candi2)
        lb_stat, lb_pvalue = lb_stat[lag_candi2-1], lb_pvalue[lag_candi2-1]
        lb_stat_all_lag2.append(lb_stat)
        lb_pvalue_all_lag2.append(lb_pvalue)
        # Ljung-Box^2 Test
        lb_stat, lb_pvalue = sm.stats.diagnostic.acorr_ljungbox(Target[col]**2, lags=lag_candi1)
        lb_stat, lb_pvalue = lb_stat[lag_candi1-1], lb_pvalue[lag_candi1-1]
        lb_stat_all_lag12.append(lb_stat)
        lb_pvalue_all_lag12.append(lb_pvalue)
        lb_stat, lb_pvalue = sm.stats.diagnostic.acorr_ljungbox(Target[col]**2, lags=lag_candi2)
        lb_stat, lb_pvalue = lb_stat[lag_candi2-1], lb_pvalue[lag_candi2-1]
        lb_stat_all_lag22.append(lb_stat)
        lb_pvalue_all_lag22.append(lb_pvalue)
        # Augmented Dickey-Fuller Test
        adf_stat, adf_pvalue = sm.tsa.stattools.adfuller(Target[col], autolag='AIC')[0], sm.tsa.stattools.adfuller(Target[col], autolag='AIC')[1]
        adf_stat_all.append(adf_stat)
        adf_pvalue_all.append(adf_pvalue)
    result = pd.concat([pd.DataFrame([Target.mean(), Target.std(), Target.skew(), Target.kurt()]),
                        pd.DataFrame(ks_stat_all, index=Target.columns).T, pd.DataFrame(ks_pvalue_all, index=Target.columns).T,
                        pd.DataFrame(jb_stat_all, index=Target.columns).T, pd.DataFrame(jb_pvalue_all, index=Target.columns).T,
                        pd.DataFrame(arch_stat_all_lag1, index=Target.columns).T, pd.DataFrame(arch_pvalue_all_lag1, index=Target.columns).T,
                        pd.DataFrame(arch_stat_all_lag2, index=Target.columns).T, pd.DataFrame(arch_pvalue_all_lag2, index=Target.columns).T,
                        pd.DataFrame(lb_stat_all_lag1, index=Target.columns).T, pd.DataFrame(lb_pvalue_all_lag1, index=Target.columns).T,
                        pd.DataFrame(lb_stat_all_lag2, index=Target.columns).T, pd.DataFrame(lb_pvalue_all_lag2, index=Target.columns).T,
                        pd.DataFrame(lb_stat_all_lag12, index=Target.columns).T, pd.DataFrame(lb_pvalue_all_lag12, index=Target.columns).T,
                        pd.DataFrame(lb_stat_all_lag22, index=Target.columns).T, pd.DataFrame(lb_pvalue_all_lag22, index=Target.columns).T,
                        pd.DataFrame(adf_stat_all, index=Target.columns).T, pd.DataFrame(adf_pvalue_all, index=Target.columns).T])
    result.index = ['Mean', 'Stdev', 'Skewness', 'Kurtosis', 'Kolmogorov–Smirnov', '(p-value)', 'Jarque–Bera', '(p-value)',
                    'ARCH(10)', '(p-value)', 'ARCH(20)', '(p-value)', 'Ljung–Box(10)', '(p-value)', 'Ljung–Box(20)', '(p-value)',
                    'Ljung–Box(10)^2', '(p-value)', 'Ljung–Box(20)^2', '(p-value)', 'Augmented Dickey–Fuller', '(p-value)']
    return result

# Summary Statistics (by Type)
def Summary_AllGroup(raw_HPI, raw_JDI, Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol, Apartments_Kcol, Detached_Houses_Kcol, Row_Houses_Kcol):
    print('Summary Statistics (All)')
    result_final = pd.DataFrame()
    result1 = summary_statistics(raw_HPI[Apartments_Scol])
    result_final = pd.concat([result_final, pd.DataFrame((result1.iloc[:4,:].T).mean(), columns=[raw_HPI[Apartments_Scol].columns[0].split(':')[1]])], axis=1)
    result2 = summary_statistics(raw_JDI[Apartments_Kcol])
    result_final = pd.concat([result_final, pd.DataFrame((result2.iloc[:4,:].T).mean(), columns=[raw_JDI[Apartments_Kcol].columns[0].split(':')[1]])], axis=1)
    result3 = summary_statistics(raw_HPI[Detached_Houses_Scol])
    result_final = pd.concat([result_final, pd.DataFrame((result3.iloc[:4,:].T).mean(), columns=[raw_HPI[Detached_Houses_Scol].columns[0].split(':')[1]])], axis=1)
    result4 = summary_statistics(raw_JDI[Detached_Houses_Kcol])
    result_final = pd.concat([result_final, pd.DataFrame((result4.iloc[:4,:].T).mean(), columns=[raw_JDI[Detached_Houses_Kcol].columns[0].split(':')[1]])], axis=1)
    result5 = summary_statistics(raw_HPI[Row_Houses_Scol])
    result_final = pd.concat([result_final, pd.DataFrame((result5.iloc[:4,:].T).mean(), columns=[raw_HPI[Row_Houses_Scol].columns[0].split(':')[1]])], axis=1)
    result6 = summary_statistics(raw_JDI[Row_Houses_Kcol])
    result_final = pd.concat([result_final, pd.DataFrame((result6.iloc[:4,:].T).mean(), columns=[raw_JDI[Row_Houses_Kcol].columns[0].split(':')[1]])], axis=1)
    display(result_final)
    print('Summary Statistics ({})'.format(raw_HPI[Apartments_Scol].columns[0].split(':')[1]))
    display(result1.iloc[:4,:].rename(columns=lambda x: x.split(':')[0]))
    print('Summary Statistics ({})'.format(raw_JDI[Apartments_Kcol].columns[0].split(':')[1]))
    display(result2.iloc[:4,:].rename(columns=lambda x: x.split(':')[0]))
    print('Summary Statistics ({})'.format(raw_HPI[Detached_Houses_Scol].columns[0].split(':')[1]))
    display(result3.iloc[:4,:].rename(columns=lambda x: x.split(':')[0]))
    print('Summary Statistics ({})'.format(raw_JDI[Detached_Houses_Kcol].columns[0].split(':')[1]))
    display(result4.iloc[:4,:].rename(columns=lambda x: x.split(':')[0]))
    print('Summary Statistics ({})'.format(raw_HPI[Row_Houses_Scol].columns[0].split(':')[1]))
    display(result5.iloc[:4,:].rename(columns=lambda x: x.split(':')[0]))
    print('Summary Statistics ({})'.format(raw_JDI[Row_Houses_Kcol].columns[0].split(':')[1]))
    display(result6.iloc[:4,:].rename(columns=lambda x: x.split(':')[0]))
    return result_final, result1, result2, result3, result4, result5, result6
   
# Index to Return
def index_to_return(Target):
    log_return_df = []
    for col in Target.columns:
        log_return = []
        log_return = (np.log(Target[col]) - np.log(Target[col].shift(1)))
        log_return[0] = log_return.mean()
        log_return_df.append(log_return)
    log_return_final = pd.DataFrame(log_return_df).T
    return log_return_final

# Summary Statistics of Return (by Type)
def Summary_Return_AllGroup(raw_HPI_return, Grouplist):
    for col in Grouplist:
        print('Summary Statistics of Return ({})'.format(raw_HPI_return[col].columns[0].split(':')[1]))
        result = summary_statistics(raw_HPI_return[col])
        display(result.rename(columns=lambda x: x.split(':')[0]))

# Calulate Conditional Volatility
def conditional_volatility(Target):
    conditional_vol = []
    for col in Target.columns:
        Target_arima = sm.tsa.ARIMA(Target[col]*100, order=(1,0,1)).fit()
        Target_garch = arch.arch_model(Target_arima.resid.values, vol='Garch', p=1, o=0, q=1, dist='generalized error').fit(disp='off')
        conditional_vol.append(Target_garch.conditional_volatility)  
    conditional_vol_final = pd.DataFrame(conditional_vol, index=Target.columns, columns=Target.index).T
    return conditional_vol_final

# Rearrange of Stochastic Parameters
def get_stochastic_parameters(Target_index, Target_return, Target_cvol, deltat=1):
    parameters_final = dict()
    for col in Target_index.columns:
        parameters = pd.concat([Target_index[col], Target_return[col], Target_return[col].rolling(window=3, center=False).mean().fillna(0),
                                Target_cvol[col]/100], axis=1)
        parameters.columns = ['Index', 'Return', 'Mu', 'CVol']
        parameters['Nu'] = parameters['Mu'] - (0.5)*(parameters['CVol']**2)
        parameters['Up_return'] = np.exp(parameters['Nu']*deltat + parameters['CVol']*deltat)
        parameters['Down_return'] = np.exp(parameters['Nu']*deltat - parameters['CVol']*deltat)
        parameters_final[col] = parameters
    return parameters_final

# Grouping of Columns
def Grouping(S_real_full):
    Apartments_Scol, Apartments_Kcol, Detached_Houses_Scol, Detached_Houses_Kcol, Row_Houses_Scol, Row_Houses_Kcol = [], [], [], [], [], [] 
    for col in S_real_full.keys():
        if 'Apartments(HPI)' in col.split(':'):
            Apartments_Scol.append(col)
            Apartments_Kcol.append(col.replace('(HPI)', '(JDI)'))
        elif 'Detached_Houses(HPI)' in col.split(':'):
            Detached_Houses_Scol.append(col)
            Detached_Houses_Kcol.append(col.replace('(HPI)', '(JDI)'))
        elif 'Row_Houses(HPI)' in col.split(':'):
            Row_Houses_Scol.append(col)
            Row_Houses_Kcol.append(col.replace('(HPI)', '(JDI)'))
    return Apartments_Scol, Apartments_Kcol, Detached_Houses_Scol, Detached_Houses_Kcol, Row_Houses_Scol, Row_Houses_Kcol

# Calculate S
def S_generator(S_real, t, T):
    S = np.full((2**(T-t), T-t+1), np.nan)
    S[0][0] = S_real['Index'].iloc[t]
    for i in range(1,T-t+1): # 1~12 (2~13)
        for j in range(0,2**(i)): # 0~2^1-1/2^12-1 (1~2^1/2^12)
            if j%2 == 0:
                S[j][i] = S[int(j/2)][i-1] * S_real['Up_return'][t+(i-1)]
            else:
                S[j][i] = S[int(j/2)][i-1] * S_real['Down_return'][t+(i-1)]
    return S

# Calculate V
def V_generator(S_real, S, K_real, R_real, t, T, early_exercise):
    ### Generate V
    V_call = np.full((S.shape[0], S.shape[1]), np.nan)
    V_put = np.full((S.shape[0], S.shape[1]), np.nan)
    initial_spread = S_real['Index'][t] - K_real[t];
    V_call_early = np.full((S.shape[0], S.shape[1]-1), np.nan)
    V_put_early = np.full((S.shape[0], S.shape[1]-1), np.nan)
    Time_call_early = 0
    Time_put_early = 0
    print(T)
    for i in range(S.shape[0]-1, -1, -1): #4095~0 (4096~1)
        V_call[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
        V_put[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
    print(S[i][S.shape[1]-1])
    print(K_real[T])
    for j in range(0, S.shape[1]-1): #0~11 (1~12)
        for i in range(0, 2**(S.shape[1]-1-(j+1))): # 0~2^11-1/2^0(1~2^11/2^1)
            V_call[i][S.shape[1]-1-(j+1)] = (0.5 * V_call[2*i][S.shape[1]-1-j] + 0.5 * V_call[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            V_put[i][S.shape[1]-1-(j+1)] = (0.5 * V_put[2*i][S.shape[1]-1-j] + 0.5 * V_put[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            ### Calculate V(when early exercise is True)
            if early_exercise:
                V_call_early[i][S.shape[1]-1-(j+1)] = S[i][S.shape[1]-1-(j+1)] - K_real[T-(j+1)] - initial_spread
                if V_call_early[i][S.shape[1]-1-(j+1)] < V_call[i][S.shape[1]-1-(j+1)]:
                    V_call[i][S.shape[1]-1-(j+1)] = V_call_early[i][S.shape[1]-1-(j+1)]
                    Time_call_early = j+1
                V_put_early[i][S.shape[1]-1-(j+1)] = S[i][S.shape[1]-1-(j+1)] - K_real[T-(j+1)] - initial_spread
                if V_put_early[i][S.shape[1]-1-(j+1)] > V_put[i][S.shape[1]-1-(j+1)]:
                    V_put[i][S.shape[1]-1-(j+1)] = V_put_early[i][S.shape[1]-1-(j+1)]
                    Time_put_early = j+1
    return V_call, V_put, Time_call_early, Time_put_early

# Calculate V
def V_generator_simple(S_real, S, K_real, R_real, t, T, early_exercise):
    ### Generate V
    V_call = np.full((S.shape[0], S.shape[1]), np.nan)
    V_put = np.full((S.shape[0], S.shape[1]), np.nan)
    initial_spread = S_real['Index'][t] - K_real[t];
    V_early = np.full((S.shape[0], S.shape[1]-1), np.nan)
    Time_call_early = 0
    Time_put_early = 0
    for i in range(S.shape[0]-1, -1, -1): #4095~0 (4096~1)
        V_call[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
        V_put[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
    for j in range(0, S.shape[1]-1): #0~11 (1~12)
        for i in range(0, 2**(S.shape[1]-1-(j+1))): # 0~2^11-1/2^0(1~2^11/2^1)
            V_call[i][S.shape[1]-1-(j+1)] = (0.5 * V_call[2*i][S.shape[1]-1-j] + 0.5 * V_call[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            V_put[i][S.shape[1]-1-(j+1)] = (0.5 * V_put[2*i][S.shape[1]-1-j] + 0.5 * V_put[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            ### Calculate V(when early exercise is True)
            if early_exercise:
                V_early[i][S.shape[1]-1-(j+1)] = S[i][S.shape[1]-1-(j+1)] - K_real[T-(j+1)] - initial_spread
                if V_early[i][S.shape[1]-1-(j+1)] < V_call[i][S.shape[1]-1-(j+1)]:
                    V_call[i][S.shape[1]-1-(j+1)] = V_early[i][S.shape[1]-1-(j+1)]
                    Time_call_early = j+1
                if V_early[i][S.shape[1]-1-(j+1)] > V_put[i][S.shape[1]-1-(j+1)]:
                    V_put[i][S.shape[1]-1-(j+1)] = V_early[i][S.shape[1]-1-(j+1)]
                    Time_put_early = j+1
    return V_call, V_put, Time_call_early, Time_put_early

# Summary of One Column Spread
def Spread_OneColumn(S_real, K_real, R_real, start, end, interval, early_exercise):
    # Checking bound of input range
    initial = (pd.to_datetime(start).year - S_real.index[0].year) * 12 + pd.to_datetime(start).month - 1
    final = (pd.to_datetime(end).year - S_real.index[0].year) * 12 + pd.to_datetime(end).month - 1
    if (final - initial) < interval:
        print('Out of Bound!!!')
    # Calculate S and V
    V_final = []
    early_call = []
    early_put = []
    for t in range(initial, final, interval):
        T = t + interval
        ### Calculate S
        S = S_generator(S_real, t, T)
        ### Calculate V
        V_call, V_put, Time_call_early, Time_put_early = V_generator_simple(S_real, S, K_real, R_real, t, T, early_exercise)
        V_final.append([V_call[0][0], V_put[0][0]])
        ### Calculate dates of early exercises
        if Time_call_early == 0:
            early_call.append(0)
        else:
            early_call.append(S_real.index[T-Time_call_early])
        if Time_put_early == 0:
            early_put.append(0)
        else:
            early_put.append(S_real.index[T-Time_put_early])
    # Results
    Time_early_exercise = pd.concat([pd.DataFrame(early_call, columns=['Early Exercise of Call']),
                                     pd.DataFrame(early_put, columns=['Early Exercise of Put'])], axis=1)
    V_final = pd.DataFrame(V_final, columns=['V(Call)', 'V(Put)'])
    return V_final, Time_early_exercise

# Summary of Full Column Spread
def Spread_FullColumns(S_real_full, K_real_full, R_real, start, end, interval, early_exercise):
    Spread_full = dict()
    Time_early_full = dict()
    R_real = R_real.copy()
    for target in tqdm(S_real_full.keys()):
        S_real = S_real_full[target]
        K_real = K_real_full[target.replace('(HPI)', '(JDI)')]
        Spread, Time_early = Spread_OneColumn(S_real, K_real, R_real, start, end, interval, early_exercise)
        Spread_full[target] = Spread
        Time_early_full[target] = Time_early
    return Spread_full, Time_early_full

# Summary of Full Result Statistics
def Summary_Result_Full(Result, Result_Summary, Grouplist):
    result_summary = pd.DataFrame()
    for each in Grouplist:
        print('Absolute Bubble Statistics in Full Period ({})'.format(each[0].split(':')[1]))
        result_table = pd.concat([abs(Result[each]).mean(), (abs(Result[each]).mean()/Result_Summary[each]*100),
                                  abs(Result[each]).max(), (abs(Result[each]).max()/Result_Summary[each]*100),
                                  abs(Result[each]).min(), (abs(Result[each]).min()/Result_Summary[each]*100)], axis=1)
        result_table.columns = ['Average', 'Average(%)', 'Max', 'Max(%)', 'Min', 'Min(%)']
        result_table.rename(index=lambda x: x.split(':')[0], inplace=True)
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table.mean(), columns=[each[0].split(':')[1]]).T], axis=0)
        result_table = result_table.style.format({'Average(%)':'({:.2f}%)', 'Max(%)':'({:.2f}%)', 'Min(%)':'({:.2f}%)'})
        display(result_table)
    result_summary = result_summary.style.format({'Average(%)':'({:.2f}%)', 'Max(%)':'({:.2f}%)', 'Min(%)':'({:.2f}%)'})
    print('Absolute Bubble Statistics in Full Period (All)')
    display(result_summary)
    result_summary = pd.DataFrame()
    for each in Grouplist:
        print('Real Bubble Statistics in Full Period ({})'.format(each[0].split(':')[1]))
        result_table = pd.concat([Result[each].mean(), Result[each].mean()/Result_Summary[each]*100,
                                  Result[each].max(), Result[each].max()/Result_Summary[each]*100,
                                  Result[each].min(), Result[each].min()/Result_Summary[each]*100], axis=1)
        result_table.columns = ['Average', 'Average(%)', 'Max', 'Max(%)', 'Min', 'Min(%)']
        result_table.rename(index=lambda x: x.split(':')[0], inplace=True)
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table.mean(), columns=[each[0].split(':')[1]]).T], axis=0)
        result_table = result_table.style.format({'Average(%)':'({:.2f}%)', 'Max(%)':'({:.2f}%)', 'Min(%)':'({:.2f}%)'})
        display(result_table)
    result_summary = result_summary.style.format({'Average(%)':'({:.2f}%)', 'Max(%)':'({:.2f}%)', 'Min(%)':'({:.2f}%)'})
    print('Real Bubble Statistics in Full Period (All)')
    display(result_summary)

# Summary of Trend Result Statistics
def Summary_Result_Trend(Result, Apartments_col, Detached_Houses_col, Row_Houses_col, windows=5):
    print('Absolute Bubble Statistics in Some periods')
    result_summary = pd.DataFrame()
    for each in [Apartments_col, Detached_Houses_col, Row_Houses_col]:
        result_table_concat = pd.DataFrame()
        for i in range(0,len(Result),windows):
            result_sub = Result.iloc[i:i+windows,:]
            result_table = pd.concat([abs(result_sub[each]).mean(), abs(result_sub[each]).max(), 
                                      (abs(result_sub[each]).max()/abs(result_sub[each]).mean()-1)*100,
                                      abs(result_sub[each]).min(),
                                      (abs(result_sub[each]).min()/abs(result_sub[each]).mean()-1)*100], axis=1)
            result_table_concat = pd.concat([result_table_concat, result_table], axis=1)
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table_concat.mean(), columns=[result_table_concat.index[0].split(":")[1]]).T], axis=0)
        result_table_concat.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                                  ['Average', 'Max', 'Max(%)', 'Min', 'Min(%)']])
        result_table_concat = result_table_concat.style.format({('2004~2008', 'Max(%)'):'({:.0f}%)', ('2004~2008', 'Min(%)'):'({:.0f}%)',
                                                                ('2009~2013', 'Max(%)'):'({:.0f}%)', ('2009~2013', 'Min(%)'):'({:.0f}%)',
                                                                ('2014~2018', 'Max(%)'):'({:.0f}%)', ('2014~2018', 'Min(%)'):'({:.0f}%)'})
        display(result_table_concat)
    result_summary.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                         ['Average', 'Max', 'Max(%)', 'Min', 'Min(%)']])
    result_summary = result_summary.style.format({('2004~2008', 'Max(%)'):'({:.0f}%)', ('2004~2008', 'Min(%)'):'({:.0f}%)',
                                                  ('2009~2013', 'Max(%)'):'({:.0f}%)', ('2009~2013', 'Min(%)'):'({:.0f}%)',
                                                  ('2014~2018', 'Max(%)'):'({:.0f}%)', ('2014~2018', 'Min(%)'):'({:.0f}%)'})
    display(result_summary)
    print('Real Bubble Statistics in Some periods')
    result_summary = pd.DataFrame()
    for each in [Apartments_col, Detached_Houses_col, Row_Houses_col]:
        result_table_concat = pd.DataFrame()
        for i in range(0,len(Result),windows):
            result_sub = Result.iloc[i:i+windows,:]
            result_table = pd.concat([result_sub[each].mean(), result_sub[each].max(), result_sub[each].min(),
                                abs(result_sub[each].min()/result_sub[each].max())*100], axis=1)
            result_table_concat = pd.concat([result_table_concat, result_table], axis=1)
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table_concat.mean(), columns=[result_table_concat.index[0].split(":")[1]]).T], axis=0)
        result_table_concat.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                                  ['Average', 'Max', 'Min', 'abs(Min/Max)(%)']])
        result_table_concat = result_table_concat.style.format({('2004~2008', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                                ('2009~2013', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                                ('2014~2018', 'abs(Min/Max)(%)'):'({:.0f}%)'})
        display(result_table_concat)
    result_summary.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                         ['Average', 'Max', 'Min', 'abs(Min/Max)(%)']])
    result_summary = result_summary.style.format({('2004~2008', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                  ('2009~2013', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                  ('2014~2018', 'abs(Min/Max)(%)'):'({:.0f}%)'})
    display(result_summary)
    
# Calculate Rowsum of Data by Windows
def rowsum_bywindow(Target, windows):
    result_rowsum = pd.DataFrame()
    for i in range(int((Target.shape[0]+1)/windows)):
        result_rowsum = pd.concat([result_rowsum, pd.DataFrame(Target.iloc[i*windows:i*windows+2,:].sum()).T], axis=0)
    result_rowsum.index = [n for n in range(i+1)]
    return result_rowsum

# Calculate Arbitrage Opportunity
def Arbitrage_Calculator(Result_12M, Result_6M):
    arbitrage_value, arbitrage_count = pd.DataFrame(), pd.DataFrame()
    for col in Result_12M.columns:
        spread_bywindow = pd.concat([Result_12M[col].shift(1), Result_6M[col].shift(1), Result_6M[col], Result_12M[col]], axis=1)
        max_TF, min_TF = pd.DataFrame(), pd.DataFrame()
        for i in range(spread_bywindow.shape[0]):
            max_TF = pd.concat([max_TF, pd.DataFrame(spread_bywindow[spread_bywindow > 0].iloc[i,:] == spread_bywindow[spread_bywindow > 0].max(axis=1)[i]).T], axis=0)
            min_TF = pd.concat([min_TF, pd.DataFrame(spread_bywindow[spread_bywindow < 0].iloc[i,:] == spread_bywindow[spread_bywindow < 0].min(axis=1)[i]).T], axis=0)
        arbitrage_count = pd.concat([arbitrage_count,
                                     pd.DataFrame([list(max_TF.sum(axis=0))[0]+list(max_TF.sum(axis=0))[3],
                                                   list(max_TF.sum(axis=0))[1]+list(max_TF.sum(axis=0))[2],
                                                   list(min_TF.sum(axis=0))[1]+list(min_TF.sum(axis=0))[2],
                                                   list(min_TF.sum(axis=0))[0]+list(min_TF.sum(axis=0))[3]],
                                                  columns=[col], index = ['Arbitrage_12M(Short)', 'Arbitrage_6M(Short)', 'Arbitrage_6M(Long)', 'Arbitrage_12M(Long)']).T], axis=0)
        arbitrage_value = pd.concat([arbitrage_value,
                                     pd.DataFrame([(list(spread_bywindow[max_TF].sum(axis=0))[0]+list(spread_bywindow[max_TF].sum(axis=0))[3]),
                                                   (list(spread_bywindow[max_TF].sum(axis=0))[1]+list(spread_bywindow[max_TF].sum(axis=0))[2]),
                                                   (list(spread_bywindow[min_TF].sum(axis=0))[1]+list(spread_bywindow[min_TF].sum(axis=0))[2]),
                                                   (list(spread_bywindow[min_TF].sum(axis=0))[0]+list(spread_bywindow[min_TF].sum(axis=0))[3])],
                                                  columns=[col], index = ['Arbitrage_12M(Short)', 'Arbitrage_6M(Short)', 'Arbitrage_6M(Long)', 'Arbitrage_12M(Long)']).T], axis=0)
    arbitrage_value = (arbitrage_value/arbitrage_count).fillna(0)
    return arbitrage_count, arbitrage_value

# Summary Arbitrage Count and Value
def Summary_Arbitrage(arbi_nonearly_count, arbi_nonearly_value, arbi_earlylong_count, arbi_earlylong_value, arbi_earlyshort_count, arbi_earlyshort_value):
    print('Summary of Arbitrage Trend in Count and Value')
    arbi_count_full = pd.DataFrame()
    arbi_count_full = pd.concat([arbi_count_full, pd.DataFrame(arbi_nonearly_count.mean(axis=0)).T])
    arbi_count_full = pd.concat([arbi_count_full, pd.DataFrame(arbi_earlylong_count.mean(axis=0)).T + pd.DataFrame(arbi_earlyshort_count.mean(axis=0)).T])
    arbi_value_full = pd.DataFrame()
    arbi_value_full = pd.concat([arbi_value_full, pd.DataFrame(arbi_nonearly_value.mean(axis=0)).T])
    arbi_value_full = pd.concat([arbi_value_full, pd.DataFrame(arbi_earlylong_value.mean(axis=0)).T + pd.DataFrame(arbi_earlyshort_value.mean(axis=0)).T])
    for col in [Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol]:
        arbi_count_full = pd.concat([arbi_count_full, pd.DataFrame(arbi_nonearly_count.loc[col].mean(axis=0)).T])
        arbi_count_full = pd.concat([arbi_count_full, pd.DataFrame(arbi_earlylong_count.loc[col].mean(axis=0)).T + pd.DataFrame(arbi_earlyshort_count.loc[col].mean(axis=0)).T])
        arbi_value_full = pd.concat([arbi_value_full, pd.DataFrame(arbi_nonearly_value.loc[col].mean(axis=0)).T])
        arbi_value_full = pd.concat([arbi_value_full, pd.DataFrame(arbi_earlylong_value.loc[col].mean(axis=0)).T + pd.DataFrame(arbi_earlyshort_value.loc[col].mean(axis=0)).T])
    arbi_count_full.index = ['All Types', 'All Types(early)', 'Apartments', 'Apartments(early)', 'Detached_Houses', 'Detached_Houses(early)', 'Row_Houses', 'Row_Houses(early)']
    arbi_value_full.index = ['All Types', 'All Types(early)', 'Apartments', 'Apartments(early)', 'Detached_Houses', 'Detached_Houses(early)', 'Row_Houses', 'Row_Houses(early)']
    display(arbi_count_full, arbi_value_full)
    display(arbi_nonearly_count, arbi_nonearly_value, arbi_earlylong_count+arbi_earlyshort_count, arbi_earlylong_value+arbi_earlyshort_value)

# Main
1) 만원단위 절삭: Index 나누기 10000  
2) Return 변환: (log(t)-log(t-1))으로 Return 변환 -> 첫줄(NaN)은 Return 평균값 대체  
3) Conditional Volatility 계산: Return*100으로 ARIMA(1,1) + GARCH(1,1) 적용  
4) Stochastic Parameters of S 정리:  
=> HPI / Return / Mu / CVol(위CVol나누기100) / Nu / Up_return / Down_return  
5) S 트리노미얼 트리 확장  
6) V 트리노미얼 트리 확장  
7) 조기행사 조건부 트리 업데이트  

In [6]:
# Data Loading
location = 'C:\\Users\\KK\\Desktop\\20190104_KB주택가격동향_KK.xlsx'
raw_HPI = pd.read_excel(location, sheet_name='Raw_HPI_value')
raw_JDI = pd.read_excel(location, sheet_name='Raw_JDI_value')
raw_IR = pd.read_excel(location, sheet_name='Raw_IR')
raw_IR.columns = ['Time', 'Interest_Rate']
raw_IR = raw_IR.iloc[3:,:]

# Indexing and Drop
raw_HPI.set_index(pd.date_range('1986-01', '2019-01', freq='M'), inplace=True)
raw_JDI.set_index(pd.date_range('1986-01', '2019-01', freq='M'), inplace=True)
raw_IR.set_index(pd.date_range('1987-01', '2019-01', freq='M'), inplace=True)
del raw_HPI['Time']
del raw_JDI['Time']
del raw_IR['Time']

# Time & Null Filter
raw_HPI = raw_HPI.loc[raw_HPI.index >= '2004-01', raw_HPI.isna().sum() == 0]
raw_JDI = raw_JDI.loc[raw_JDI.index >= '2004-01', raw_JDI.isna().sum() == 0]
raw_JDI.loc['2019-01-31'] = raw_JDI.iloc[len(raw_JDI)-1]
raw_IR = raw_IR[raw_IR.index >= '2004-01'] / (100*12) # Transfer to Monthly IR
raw_IR = pd.DataFrame(raw_IR.Interest_Rate.astype('float'))
raw_IR.loc['2019-01-31'] = raw_IR.values.flatten()[len(raw_IR)-1]

# Index to Return
raw_HPI_return = index_to_return(raw_HPI)

# Calulate Conditional Volatility
raw_HPI_cvol = conditional_volatility(raw_HPI_return)

# Rearrange of Stochastic Parameters
raw_HPI_parameters = get_stochastic_parameters(raw_HPI, raw_HPI_return, raw_HPI_cvol)

# Grouping of Columns
Apartments_Scol, Apartments_Kcol, Detached_Houses_Scol, Detached_Houses_Kcol, Row_Houses_Scol, Row_Houses_Kcol = Grouping(raw_HPI_parameters)

# Calculate S & V value
# Spread, Time_early = Spread_OneColumn(raw_HPI_parameters['Seoul:Apartments(HPI)'], raw_JDI['Seoul:Apartments(JDI)'], raw_IR,
#                                       start = '2004-01', end = '2018-12', interval = 12, early_exercise = False)
print('Processing of spread calculation without early exercise')
print('Trading by 12 months')
Spreadfull, Time_early_full = Spread_FullColumns(raw_HPI_parameters, raw_JDI, raw_IR,
                                                 start = '2004-01', end = '2018-12', interval = 12, early_exercise = False)
print('Trading by 6 months')
Spreadfull_by6, Time_early_full_by6 = Spread_FullColumns(raw_HPI_parameters, raw_JDI, raw_IR,
                                                 start = '2004-01', end = '2018-12', interval = 6, early_exercise = False)
print('Processing of spread calculation with early exercise')
print('Trading by 12 months')
Spreadfull_early, Time_early_full = Spread_FullColumns(raw_HPI_parameters, raw_JDI, raw_IR,
                                                 start = '2004-01', end = '2018-12', interval = 12, early_exercise = True)
print('Trading by 6 months')
Spreadfull_early_by6, Time_early_full_by6 = Spread_FullColumns(raw_HPI_parameters, raw_JDI, raw_IR,
                                                 start = '2004-01', end = '2018-12', interval = 6, early_exercise = True)

Processing of spread calculation without early exercise
Trading by 12 months


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [03:13<00:00,  7.72s/it]


Trading by 6 months


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:06<00:00,  4.02it/s]


Processing of spread calculation with early exercise
Trading by 12 months


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [03:54<00:00,  9.40s/it]


Trading by 6 months


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:07<00:00,  3.34it/s]


# Summary Result

In [9]:
# Summary Statistics (by all and types)
Summary_All, Summary_HPI_apa, Summary_JDI_apa, Summary_HPI_det, Summary_JDI_det, Summary_HPI_row, Summary_JDI_row = Summary_AllGroup(raw_HPI, raw_JDI,
                                                                                                                                     Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol,
                                                                                                                                     Apartments_Kcol, Detached_Houses_Kcol, Row_Houses_Kcol)
result_IR = summary_statistics(raw_IR*100*12) # For the Transfered Yearly IR
print('Summary Statistics ({})'.format(raw_IR.columns[0]))
display(result_IR.iloc[:4,:])

# Summary Statistics of HPI Returns
Summary_Return_AllGroup(raw_HPI_return, Grouplist=[Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol])

# Arrange of Results
result, result_by6 = pd.DataFrame(), pd.DataFrame()
for key in Spreadfull.keys():
    result = pd.concat([result, Spreadfull[key]['V(Call)']], axis=1)
    result_by6 = pd.concat([result_by6, Spreadfull_by6[key]['V(Call)']], axis=1)
result_by6 = rowsum_bywindow(result_by6, 2)
result.columns, result_by6.columns = Spreadfull.keys(), Spreadfull_by6.keys()
result_call, result_call_by6 = pd.DataFrame(), pd.DataFrame()
for key in Spreadfull_early.keys():
    result_call = pd.concat([result_call, Spreadfull_early[key]['V(Call)']], axis=1)
    result_call_by6 = pd.concat([result_call_by6, Spreadfull_early_by6[key]['V(Call)']], axis=1)
result_call_by6 = rowsum_bywindow(result_call_by6, 2)
result_call.columns, result_call_by6.columns = Spreadfull_early.keys(), Spreadfull_early_by6.keys()
result_put, result_put_by6 = pd.DataFrame(), pd.DataFrame()
for key in Spreadfull_early.keys():
    result_put = pd.concat([result_put, Spreadfull_early[key]['V(Put)']], axis=1)
    result_put_by6 = pd.concat([result_put_by6, Spreadfull_early_by6[key]['V(Put)']], axis=1)
result_put_by6 = rowsum_bywindow(result_put_by6, 2)
result_put.columns, result_put_by6.columns = Spreadfull_early.keys(), Spreadfull_early_by6.keys()

# Bubble Statistics
Summary_HPI = pd.concat([Summary_HPI_apa.loc['Mean'], Summary_HPI_det.loc['Mean'], Summary_HPI_row.loc['Mean']], axis=0)
Summary_Result_Full(result, Summary_HPI, Grouplist=[Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol])

# Bubble Statistics (by trend)
Summary_Result_Trend(result, Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol, windows=5)

# Bubble Statistics when early exercises
Summary_Result_Full(result_call, Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol)
Summary_Result_Full(result_put, Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol)

# Bubble Statistics when early exercises (by trend)
Summary_Result_Trend(result_call, Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol, windows=5)
Summary_Result_Trend(result_put, Apartments_Scol, Detached_Houses_Scol, Row_Houses_Scol, windows=5)

# Arbitrage Opportunity
arbi_nonearly_count, arbi_nonearly_value = Arbitrage_Calculator(result, result_by6)
arbi_earlylong_count, arbi_earlylong_value = Arbitrage_Calculator(result_call, result_call_by6)
arbi_earlyshort_count, arbi_earlyshort_value = Arbitrage_Calculator(result_put, result_put_by6)
Summary_Arbitrage(arbi_nonearly_count, arbi_nonearly_value, arbi_earlylong_count, arbi_earlylong_value, arbi_earlyshort_count, arbi_earlyshort_value)

Summary Statistics (All)


,Apartments(HPI),Apartments(JDI),Detached_Houses(HPI),Detached_Houses(JDI),Row_Houses(HPI),Row_Houses(JDI)
Mean,"27,504.10","16,792.87","49,197.73","20,566.24","17,783.43","10,873.09"
Stdev,"3,957.08","3,966.40","5,068.73","2,525.89","2,183.69","1,979.84"
Skewness,-0.26,0.08,-0.50,0.03,-0.85,-0.03
Kurtosis,-1.04,-1.64,-0.59,-1.35,-0.57,-1.43


Summary Statistics (Apartments(HPI))


,Seoul,Northern Seoul,Southern Seoul,6 Large Cities,Gyeonggi-do,Gangwon-do,Chungcheongbuk-do,Chungcheongnam-do,Jeollabuk-do,Jeollanam-do,Gyeongsangbuk-do,Gyeongsangnam-do,Non-Metropolitan Area
Mean,"62,638.21","46,736.65","76,030.43","21,739.38","31,545.54","14,406.90","13,389.09","15,676.47","14,939.69","13,982.16","13,911.59","17,121.54","15,435.61"
Stdev,"7,878.61","6,149.53","9,351.33","3,960.62","3,408.66","2,107.30","2,797.54","2,240.56","2,827.86","1,948.61","2,529.06","3,457.71","2,784.64"
Skewness,-0.68,-0.74,-0.61,0.08,-1.35,0.14,0.06,0.20,-0.33,-0.21,0.28,-0.13,-0.06
Kurtosis,0.50,-0.06,0.90,-1.62,0.56,-1.69,-1.75,-1.80,-1.62,-1.73,-1.55,-1.85,-1.80


Summary Statistics (Apartments(JDI))


...

Real Bubble Statistics in Full Period (All)


,Average,Average(%),Max,Max(%),Min,Min(%)
Apartments(HPI),208.559,(0.45%),3014.33,(7.97%),-1539.26,(-4.32%)
Detached_Houses(HPI),746.91,(1.24%),2577.54,(4.53%),-625.209,(-1.17%)
Row_Houses(HPI),87.722,(0.45%),1226.99,(6.37%),-633.739,(-3.29%)


Absolute Bubble Statistics in Some periods


Real Bubble Statistics in Some periods


TypeError: Summary_Result_Full() takes 3 positional arguments but 4 were given